In [1]:
import os
from os import path
import pandas as pd

### Build results dataframe

In [2]:
N = 24
tls = [0.5, 0.6, 0.7]
seeds = [i for i in range(1, 4)]
steps = [(i + 1) * 5000 for i in range(N)]

In [3]:
results_list_tl = []
performance_path_tl = '/misc/vlgscratch4/LecunGroup/nvidia-collab/yairschiff/pytorch-PPUU/models/planning_results'
name_tl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=False-learnedcost=False-lambdatl={tl}' + \
       '-seed={seed}-novaluestep{step}.model.log'

for tl in tls:
    for seed in seeds:
        for step in steps:
            file_name = path.join(performance_path_tl, name_tl.format(tl=tl, seed=seed, step=step))
            if os.path.isfile(file_name):
                with open(file_name) as f:
                    lines = f.readlines()
                    for line in lines:
                        splits = line.split("|")
                        ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                        time = int(splits[1].split(":")[1].strip())
                        distance = int(splits[2].split(":")[1].strip())
                        success_or_failure = int(splits[3].split(":")[1].strip())
                        df_line = {
                            'lambda_tl': tl,
                            'seed': seed,
                            'step': step,
                            'ep': ep,
                            'time': time,
                            'distance': distance,
                            'result': success_or_failure
                        }
                        results_list_tl.append(df_line)

In [13]:
results_df_tl = pd.DataFrame(results_list_tl)
cols_tl = ['lambda_tl', 'seed', 'step', 'ep', 'time', 'distance', 'result']
results_df_tl = results_df_tl[cols_tl]

In [32]:
results_list_bl = []
performance_path_bl = '{/misc/vlgscratch4/LecunGroup/}nvidia-collab/yairschiff/pytorch-PPUU/' + \
                      'models_baseline/planning_results'
name_bl = 'MPUR-policy-deterministic-model=vae-zdropout=0.5-nfeature=256-bsize=6-npred=30-ureg=0.05-lambdal=0.2' + \
       '-lambdaa=0.0-gamma=0.99-lrtz=0.0-updatez=0-inferz=0-learnedcost=False' + \
       '-seed={seed}-novaluestep{step}.model.log'
for seed in seeds:
    for step in steps:
        file_name = path.join(performance_path_bl, name_bl.format(seed=seed, step=step))
        if os.path.isfile(file_name):
            with open(file_name) as f:
                lines = f.readlines()
                for line in lines:
                    splits = line.split("|")
                    ep = int(splits[0].split(":")[-1].split("/")[0].strip())
                    time = int(splits[1].split(":")[1].strip())
                    distance = int(splits[2].split(":")[1].strip())
                    success_or_failure = int(splits[3].split(":")[1].strip())
                    df_line = {
                        'seed': seed,
                        'step': step,
                        'ep': ep,
                        'time': time,
                        'distance': distance,
                        'result': success_or_failure
                    }
                    results_list_bl.append(df_line)

In [33]:
results_df_bl = pd.DataFrame(results_list_bl)
cols_bl = ['seed', 'step', 'ep', 'time', 'distance', 'result']
results_df_bl = results_df_bl[cols_bl]

### Filter dataframe

In [50]:
# Set filter variables for target_lane results
tl = 0.6
seed = 2
step = 80000
result = 1  # 1 for successes, 0 for failures

In [51]:
# Perform filter on target_lane results
print(f'List of {"success" if result else "failure"} episodes for lambda_tl: {tl}, seed: {seed}, step: {step}:')
filtered_list_tl = results_df_tl[(results_df_tl.lambda_tl == tl) &
                                 (results_df_tl.seed == seed) &
                                 (results_df_tl.step == step) &
                                 (results_df_tl.result == result)]['ep'].values
print(len(filtered_list_tl))
print(filtered_list_tl)

List of success episodes for lambda_tl: 0.6, seed: 2, step: 80000:
442
[  1   3   4   5   6   7   8   9  10  11  12  13  16  17  18  19  20  21
  22  23  24  26  27  28  30  32  34  35  36  37  38  40  42  43  45  47
  48  49  51  52  53  54  55  56  57  58  59  61  62  63  64  65  66  67
  68  69  70  71  72  74  75  76  78  79  80  81  82  83  85  86  87  88
  89  90  91  92  94  96  97  98 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 117 120 121 123 124 126 127 128 129 130 133 134
 138 139 140 141 142 144 145 146 147 148 149 150 153 154 155 156 157 158
 159 160 161 162 163 165 166 168 169 173 174 175 176 177 178 180 182 183
 184 185 186 187 188 189 190 191 193 194 195 196 197 198 199 200 201 202
 204 205 206 207 209 210 211 212 213 215 216 218 219 220 221 222 223 224
 228 230 231 232 235 236 237 238 240 241 242 243 244 245 246 247 248 249
 250 251 252 253 254 256 257 258 259 260 261 263 264 265 266 267 268 269
 270 271 273 274 275 277 279 282 283 284 285 286 287 

In [43]:
# Set filter variables for baseline results
seed = 1
step = 115000
result = 0  # 1 for successes, 0 for failures

In [45]:
# Perform filter on baseline results
print(f'List of {"success" if result else "failure"} episodes for baseline - seed: {seed}, step: {step}:')
filtered_list_bl = results_df_bl[(results_df_bl.seed == seed) &
                                 (results_df_bl.step == step) &
                                 (results_df_bl.result == result)]['ep'].values
print(len(filtered_list_bl))
print(filtered_list_bl)

List of failure episodes for baseline - seed: 1, step: 115000:
138
[  2   5  14  15  19  25  31  33  38  39  43  46  51  55  60  71  73  77
  81  83  85  95  99 100 111 116 118 122 124 125 131 136 152 164 170 171
 172 179 181 192 197 203 208 214 215 217 222 225 229 233 234 236 239 246
 248 250 255 262 264 272 276 278 280 281 283 289 298 300 305 306 307 315
 317 323 324 332 333 335 336 342 345 354 356 366 367 374 384 385 388 389
 390 391 400 404 408 409 410 412 416 417 418 423 424 433 437 439 440 441
 448 449 451 456 459 462 464 465 471 472 473 476 479 482 488 498 509 517
 519 521 524 525 526 528 532 534 536 544 550 561]
